In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe
from functions import CreatDataGBM
import xgboost as xgb

In [2]:
X_train,Y_train,X_val,Y_val,X_test,Y_test,Weight = CreatDataGBM()

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [50]:
def NWRMSLE_exp(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.exp(preds)
    temp1 = preds/(preds+1)
    temp2 = np.log(preds+1) - labels
    return temp2*temp1, temp1*temp1 + temp2*temp1*(1-temp1)

In [51]:
def NWRMSLE_relu(preds, dtrain):
    labels = dtrain.get_label()
    preds2 = np.maximum(preds,0)
    inc = preds>0
    temp1 = inc/(1+preds2)
    temp2 = np.log(preds2+1) - labels
    sq = temp1*temp1
    return temp2*temp1, sq - temp2*sq

In [47]:
'''def NWRMSLE_sq(preds, dtrain):
    labels = dtrain.get_label()
    preds2 = preds**2
    temp1 = preds/(1+preds2)
    temp2 = np.log(preds2+1) - labels
    return temp2*temp1, temp1*temp1 + temp2*(3*preds2+1)/(preds2+1)**2'''

In [52]:
objective_fun = [(NWRMSLE_exp,np.exp),(NWRMSLE_relu,lambda x:np.maximum(x,0))]

In [9]:
def loss_func(Weight,yhat,y):
    return np.sqrt(np.sum(Weight*(np.log(yhat+1)-y)**2)/np.sum(Weight))

In [43]:
param = {}
param['eta'] = 0.25
param['max_depth'] = 10
param['min_child_weight'] = 1
param['tree_method'] = 'hist'
monitor = 20
monitorTimes = 7

In [ ]:
def hyperSearchGBM(paras):   
    model_para = param.copy()
    model_para['subsample'] = paras[0]
    model_para['colsample_bytree'] = paras[1]
    model_para['gamma'] = paras[2]
    obj,link = objective_fun[paras[3]]
    tot_loss = 0
    for i in range(16):
        dtrain = xgb.DMatrix(
            X_train, label=Y_train[:, i],
            weight=np.tile(Weight,16)
        )
        dval = xgb.DMatrix(
            X_val, label=Y_val[:, i],
            weight=Weight)
        
        best_loss = 100
        for j in range(monitorTimes):           
            # train
            if j == 0:
                model = xgb.train(param, dtrain, monitor,obj=obj)
            else:
                model = xgb.train(param, dtrain, monitor,obj=NWRMSLE_relu,xgb_model=model)
            # eval
            yhat = link(model.predict(dval,output_margin=True))
            loss = loss_func(Weight,yhat,Y_val[:, i])
            if loss < best_loss:
                best_loss = loss
            else:
                break # early stop
        tot_loss = tot_loss + best_loss
    
    tot_loss = tot_loss/16    
    print "loss:{} , subsample:{}, colsample:{}, gamma:{} ,objective:{} \n"\
              .format(tot_loss,paras[0],paras[1],paras[2],paras[3])  
    return tot_loss

In [ ]:
space = [hp.uniform('subsample',0.01,0.15),\
         hp.uniform('colsample',0.1,0.8),\
         hp.uniform('gamma',1,100),\
         hp.choice('models',[0,1])]

In [ ]:
best = fmin(hyperSearch2,space,tpe.suggest,60)